In [3]:
import torch

In [61]:
CLASS_NUMBER = 11
BATCH_SIZE = 12
SEQUENCE_LEN = 10

# embedding onehot이 0~3 을 랜덤으로 생성한다
input_tensor = torch.randint(0, CLASS_NUMBER, (BATCH_SIZE, SEQUENCE_LEN, 6))

# tensor size 확인 (batch_size, seq_len, 6)
input_tensor.size() # >>> torch.Size([12, 10, 6])

torch.Size([12, 10, 6])

In [5]:
input_tensor

tensor([[[5, 3, 0, 5, 2, 2],
         [3, 1, 4, 1, 4, 2],
         [0, 7, 2, 6, 6, 5],
         [5, 6, 1, 0, 7, 0],
         [6, 7, 4, 5, 7, 2],
         [2, 0, 7, 0, 1, 4],
         [1, 4, 1, 2, 3, 3],
         [5, 0, 4, 1, 5, 5],
         [0, 4, 6, 7, 4, 3],
         [7, 5, 1, 7, 3, 2]],

        [[7, 2, 3, 3, 7, 7],
         [5, 6, 6, 3, 1, 4],
         [1, 1, 6, 6, 2, 0],
         [6, 1, 4, 6, 2, 2],
         [7, 4, 7, 2, 3, 3],
         [0, 3, 1, 3, 0, 0],
         [6, 1, 4, 3, 1, 6],
         [1, 4, 6, 0, 2, 3],
         [4, 5, 0, 2, 7, 7],
         [4, 1, 4, 6, 1, 6]],

        [[7, 7, 7, 2, 4, 7],
         [3, 3, 1, 3, 2, 5],
         [7, 4, 1, 1, 7, 7],
         [2, 0, 2, 3, 1, 0],
         [2, 0, 1, 5, 2, 4],
         [1, 6, 6, 7, 3, 1],
         [1, 1, 6, 5, 4, 4],
         [0, 2, 0, 5, 0, 0],
         [7, 1, 4, 2, 4, 7],
         [4, 5, 6, 3, 1, 5]],

        [[6, 3, 5, 5, 6, 6],
         [3, 1, 1, 5, 4, 7],
         [6, 0, 3, 6, 0, 3],
         [2, 3, 2, 0, 7, 5],
        

In [6]:
# embedding 선언 (embedding max값, embed_dim)
embedding = torch.nn.Embedding(CLASS_NUMBER, 100)

# embedding 생성
embedding_tensor = embedding(input_tensor)
# tensor size 확인 (batch_size, seq_len, 6, embed_dim)
embedding_tensor.size() # >>> torch.Size([12, 10, 6, 100])

torch.Size([12, 10, 6, 100])

In [7]:
embedding_tensor

tensor([[[[-1.0663, -0.3826, -0.8024,  ...,  0.1620, -1.5806, -0.6838],
          [-0.8935,  0.4409,  0.2324,  ...,  0.0743, -0.8539, -1.2142],
          [ 0.1181,  1.6771,  0.5077,  ..., -0.5421, -1.7466, -0.5671],
          [-1.0663, -0.3826, -0.8024,  ...,  0.1620, -1.5806, -0.6838],
          [ 0.7787,  1.3764, -1.5493,  ..., -1.2452,  1.0228,  0.8108],
          [ 0.7787,  1.3764, -1.5493,  ..., -1.2452,  1.0228,  0.8108]],

         [[-0.8935,  0.4409,  0.2324,  ...,  0.0743, -0.8539, -1.2142],
          [ 1.0668,  0.1609, -0.4067,  ..., -1.3819, -1.7167, -0.2176],
          [ 0.4234, -0.0327,  1.7739,  ..., -0.4352,  0.5715, -0.8454],
          [ 1.0668,  0.1609, -0.4067,  ..., -1.3819, -1.7167, -0.2176],
          [ 0.4234, -0.0327,  1.7739,  ..., -0.4352,  0.5715, -0.8454],
          [ 0.7787,  1.3764, -1.5493,  ..., -1.2452,  1.0228,  0.8108]],

         [[ 0.1181,  1.6771,  0.5077,  ..., -0.5421, -1.7466, -0.5671],
          [ 0.7456,  0.0187,  0.7457,  ..., -0.0423, -0.3876

In [21]:
import torch.nn as nn
from torch.autograd import Variable


class SimpleLSTM(nn.Module):
    def __init__(
            self,
            embedding_length,
            hidden_size,
            output_size
    ):
        super().__init__()
        self.embedding_length = embedding_length
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lstm = nn.LSTM(embedding_length, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)


    def forward(self, input):
        batch_size = input.size(0)
        print(input.shape)
        h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
        c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
        print(h_0.shape, c_0.shape)
        output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        print(output.shape)
        output = self.linear(output)
        return output


In [22]:
model = SimpleLSTM(embedding_length=100, hidden_size=128, output_size=CLASS_NUMBER)
model

SimpleLSTM(
  (lstm): LSTM(100, 128)
  (linear): Linear(in_features=128, out_features=8, bias=True)
)

In [23]:
model(input_tensor)

torch.Size([12, 10, 6])
torch.Size([1, 12, 128]) torch.Size([1, 12, 128])


RuntimeError: input.size(-1) must be equal to input_size. Expected 100, got 6

In [92]:
import numpy as np
import random

N = 1000
BATCH_SIZE = 16

input_tensor = torch.from_numpy(np.random.randn(BATCH_SIZE, N, 6))
input_tensor.shape

torch.Size([16, 1000, 6])

In [93]:
input_tensor

tensor([[[ 1.3334,  0.0900,  0.9695,  2.2554, -0.8541, -0.3263],
         [ 0.5470, -0.1725, -2.0678,  1.0781, -0.9950,  0.5495],
         [ 0.5056,  0.4921,  0.6030,  1.9122, -0.9606,  0.1736],
         ...,
         [-1.4324,  0.7545,  0.2106, -0.2328, -0.1378, -0.4016],
         [-1.0029, -0.0336, -0.6210, -0.7797,  1.4970, -1.3926],
         [ 1.1494,  0.7634, -1.2224, -0.7693, -0.0620, -0.9935]],

        [[ 0.1316, -1.4035,  0.7420,  1.2231, -2.1166,  0.5537],
         [ 1.1525, -0.2788, -0.9909, -0.0120, -0.1188, -0.5925],
         [-0.2347, -0.8860, -0.7605, -0.7242,  0.6367,  0.3241],
         ...,
         [ 0.2209,  0.6896,  0.7259, -0.8754, -0.8024, -0.0131],
         [-0.4730, -0.4438, -1.2718, -1.1187,  0.5256, -1.0091],
         [-1.3298,  0.2243, -0.2588,  0.6377,  1.8225, -0.4287]],

        [[ 0.7548,  0.4390,  2.0778, -1.1392, -0.4453,  0.0216],
         [-0.3449,  0.0689, -0.2040, -0.0097,  0.9132,  0.1264],
         [-0.3837,  0.2758, -0.3398,  0.4550, -0.0361, -0.

In [94]:
input_tensor.shape

torch.Size([16, 1000, 6])

In [97]:
HIDDEN_DIM = 8

input_tensor = input_tensor.float().cuda()

lstm = nn.LSTM(input_size=6, hidden_size=HIDDEN_DIM).cuda()

h_0 = Variable(torch.zeros(1, N, HIDDEN_DIM).cuda())
c_0 = Variable(torch.zeros(1, N, HIDDEN_DIM).cuda())

output, (final_hidden_state, final_cell_state) = lstm(input_tensor, (h_0, c_0))

output # (batch_size, N, hidden_dim)

tensor([[[ 0.0882, -0.0888, -0.1193,  ...,  0.1104,  0.3252, -0.0288],
         [ 0.0032, -0.0716,  0.2534,  ..., -0.0874, -0.1426, -0.1064],
         [ 0.0012, -0.0754, -0.0522,  ...,  0.0983,  0.2599, -0.0454],
         ...,
         [-0.0395, -0.0619,  0.0749,  ..., -0.0808, -0.0118,  0.1038],
         [ 0.1853, -0.1070,  0.0817,  ..., -0.2306, -0.0620,  0.1535],
         [ 0.0720, -0.1495,  0.2136,  ..., -0.0501,  0.0146, -0.0718]],

        [[ 0.0119,  0.0124, -0.2695,  ...,  0.1151,  0.2413, -0.0732],
         [ 0.1060, -0.1177,  0.2300,  ..., -0.1368, -0.1111, -0.1700],
         [ 0.0545, -0.0724,  0.1009,  ..., -0.1397,  0.0062, -0.1581],
         ...,
         [-0.0619,  0.0419,  0.1598,  ..., -0.0368,  0.1139,  0.0903],
         [ 0.1696, -0.1375,  0.1910,  ..., -0.2479, -0.1425,  0.1355],
         [ 0.2747, -0.2192,  0.2524,  ..., -0.2561, -0.0686, -0.0575]],

        [[-0.0307,  0.1661, -0.0526,  ...,  0.1689,  0.2001,  0.0195],
         [ 0.1757, -0.1866,  0.2663,  ..., -0

In [98]:
output.shape

torch.Size([16, 1000, 8])

In [99]:
final_hidden_state.shape

torch.Size([1, 1000, 8])

In [100]:
HIDDEN_FC_DIM = 128
OUTPUT_CLASSES = 6

fc1 = nn.Linear(in_features=HIDDEN_DIM, out_features=HIDDEN_FC_DIM, device='cuda:0')
fc2 = nn.Linear(in_features=HIDDEN_FC_DIM, out_features=OUTPUT_CLASSES, device='cuda:0')

print(fc1)
print(fc2)

Linear(in_features=8, out_features=128, bias=True)
Linear(in_features=128, out_features=6, bias=True)


In [101]:
output_fc1 = fc1(final_hidden_state.cuda())
output_fc2 = fc2(output_fc1)
output_fc2

tensor([[[-0.1938,  0.0631,  0.1251,  0.0869,  0.0924, -0.1401],
         [-0.2406,  0.1020,  0.0793,  0.1855,  0.0570, -0.0394],
         [-0.2544,  0.1122,  0.0844,  0.1206, -0.0222, -0.0503],
         ...,
         [-0.2476,  0.1166,  0.0931,  0.1367,  0.0669, -0.0792],
         [-0.2534,  0.1092,  0.1231,  0.1111,  0.0137, -0.0773],
         [-0.2388,  0.1135,  0.0777,  0.0919,  0.0107, -0.1263]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [102]:
output_fc2.shape

torch.Size([1, 1000, 6])

In [113]:
output_fc2[0]

tensor([[-0.1938,  0.0631,  0.1251,  0.0869,  0.0924, -0.1401],
        [-0.2406,  0.1020,  0.0793,  0.1855,  0.0570, -0.0394],
        [-0.2544,  0.1122,  0.0844,  0.1206, -0.0222, -0.0503],
        ...,
        [-0.2476,  0.1166,  0.0931,  0.1367,  0.0669, -0.0792],
        [-0.2534,  0.1092,  0.1231,  0.1111,  0.0137, -0.0773],
        [-0.2388,  0.1135,  0.0777,  0.0919,  0.0107, -0.1263]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [119]:
import torch.nn.functional as F

F.softmax(output_fc2[0])

C:\Users\joona\AppData\Local\Temp\ipykernel_40856\1574591000.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(output_fc2[0])


tensor([[0.1355, 0.1752, 0.1864, 0.1794, 0.1804, 0.1430],
        [0.1268, 0.1786, 0.1746, 0.1942, 0.1707, 0.1551],
        [0.1284, 0.1852, 0.1802, 0.1868, 0.1619, 0.1575],
        ...,
        [0.1271, 0.1830, 0.1787, 0.1867, 0.1741, 0.1504],
        [0.1277, 0.1835, 0.1860, 0.1838, 0.1668, 0.1523],
        [0.1318, 0.1874, 0.1808, 0.1834, 0.1691, 0.1475]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [121]:
# softmax
odds = np.exp([-0.1938,  0.0631,  0.1251,  0.0869,  0.0924, -0.1401])
odds / sum(odds)

array([0.13551765, 0.17521291, 0.18641994, 0.179433  , 0.1804226 ,
       0.14299389])

In [122]:
sum([0.13551765, 0.17521291, 0.18641994, 0.179433  , 0.1804226 , 0.14299389])

0.9999999900000001

In [112]:
torch.argmax(output_fc2[0], dim=1)

tensor([2, 3, 3, 3, 2, 3, 2, 3, 3, 1, 3, 2, 3, 2, 1, 3, 3, 2, 2, 3, 2, 1, 2, 2,
        3, 2, 2, 1, 3, 3, 2, 1, 3, 3, 2, 2, 3, 3, 1, 3, 2, 3, 1, 3, 1, 1, 1, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 1, 2, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 1, 4, 2, 2,
        3, 3, 1, 3, 2, 1, 1, 1, 1, 3, 3, 2, 2, 3, 3, 3, 2, 2, 2, 1, 3, 2, 1, 3,
        3, 2, 1, 3, 1, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 1, 2, 2, 3, 3, 3, 2, 3, 3,
        2, 2, 1, 2, 1, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 2, 3, 1, 3, 3, 2, 3, 1, 2,
        2, 3, 1, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 2, 3, 1, 2, 2, 3, 2, 3,
        2, 2, 3, 2, 2, 3, 3, 1, 1, 2, 2, 2, 2, 3, 1, 2, 1, 3, 3, 1, 3, 2, 1, 3,
        2, 3, 2, 2, 3, 1, 3, 2, 2, 1, 2, 3, 3, 3, 3, 3, 2, 3, 2, 2, 3, 1, 3, 2,
        3, 3, 2, 3, 2, 3, 1, 2, 3, 3, 3, 2, 1, 3, 2, 3, 3, 2, 1, 2, 3, 2, 3, 3,
        1, 3, 2, 3, 1, 3, 3, 3, 3, 3, 2, 3, 1, 2, 2, 3, 2, 2, 3, 2, 1, 1, 3, 2,
        3, 2, 1, 2, 3, 3, 2, 3, 2, 2, 2, 3, 1, 3, 3, 1, 3, 2, 3, 3, 2, 1, 2, 2,
        2, 3, 3, 3, 3, 1, 2, 2, 3, 3, 2,